In [4]:
from PIL import Image
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import json
import os

In [5]:
font_path = 'BlackHanSans-Regular.ttf'
circle_mask = np.array(Image.open("circle.png"))
kaist_mask = np.array(Image.open('KAIST.JPG'))

In [8]:
wc_circle = WordCloud(font_path=font_path, background_color="white", mask=circle_mask, width=800, height=800)
wc_kaist = WordCloud(font_path=font_path, background_color="white", mask=kaist_mask, width=1000, height=500)

In [ ]:
for file in os.listdir('json_file'):
    words = {}
    file_dir = 'json_file/' + file
    with open(file_dir) as json_file:
        json_data = json.load(json_file)
        for word in json_data:
            keyword = word['keyword']
            score = word['score']
            words[keyword] = score
    circle = wc_circle.generate_from_frequencies(words)
    kaist = wc_kaist.generate_from_frequencies(words)
    fig = plt.figure(figsize=(10, 10))
    plt.axis("off")
    plt.imshow(circle, interpolation="bilinear")
    fig.savefig('wordcloud/'+file+'_circle.png')
    plt.imshow(kaist, interpolation="bilinear")
    fig.savefig('wordcloud/'+file+'_kaist.png')
    plt.close(fig)

In [26]:
# TEST

words = {'먹다':5, "자다":10, '입다': 7, '약속': 3, '기대': 8, '데이트': 5}
circle = wc_circle.generate_from_frequencies(words)
kaist = wc_kaist.generate_from_frequencies(words)
fig = plt.figure(figsize=(10, 10))
plt.imshow(circle, interpolation="bilinear")
plt.axis('off')
fig.savefig('test_circle.png')
plt.imshow(kaist, interpolation="bilinear")
fig.savefig('test_kaist.png')
plt.close(fig)